# Frequency Down Converter Hardware Description

In [53]:
sv_template = """

// Angle representation is always a 0.bbbbbb of a circle
// 100000 -> half circle (pi)
// 111111 -> 2*pi (almost)


module FDC
#(
    parameter BW = 12,
    parameter ABW = 8,
)
(
    input clk_fs,  //  Sampling clock
    input clk_fso2,  // Decimation clock fsample/2
    input clk_fso4,  // Decimation clock fsample/4
    input rstb, // Reset

    input [BW-1:0] Iin,
    input [BW-1:0] Qin,
    input [ABW-1:0] Wif,
    output [BW-1:0] Iout,
    output [BW-1:0] Qout
)
    reg  [BW-1:0] Iin_s, Qin_s;
    wire  [BW-1:0] Igr, Qgr;

    // Sample
    always_ff @(posedge clk_fs or negedge rstb)
        if (rstb == 0)
         begin
            Iin_s <= 'd0;
            Qin_s <= 'd0;
         end
        else
         begin
            Iin_s <= Iin;
            Qin_s <= Qin;
         end

    
    // Compute rotation angle theta 
    reg [ABW-1:0] theta; 

    always_ff @(posedge clk_fs or negedge rstb)
        if (rstb==0)
            theta <= 0;
        else
            theta <= $unsigned(theta) + $unsigned(Wif);

    // Gross Rotation
    always_comb
        case(theta[ABW-1:ABW-2])
            2'b00    :  {Igr,Qgr} = {Iin_s,Qin_s}; // rot 0
            2'b01    :  {Igr,Qgr} = {Qin_s,-$signed(Iin_s)}; //rot p/2
            2'b10    :  {Igr,Qgr} = {-$signed(Iin_s),-$signed(Qin_s)}; //rot p
            2'b11    :  {Igr,Qgr} = {-$signed(Qin_s),Iin_s}; // rot 3pi/2
        endcase

    // Fine Rotation
    // Define the angle function
    function [ABW-1:0] tangle;
        input [3:0] i;
        begin
            case(i)
            JL_FILL_TABLE
            endcase
        end
    endfunction

    wire signed [BW-1:0] x [JL_IT_NUM-1:0];
    wire signed [BW-1:0] y [JL_IT_NUM-1:0];
    wire signed [ABW-1:0] z [JL_IT_NUM-1:0];

    assign x[0] = Igr;
    assign y[0] = Qgr;
    assign z[0] = theta;
    wire Ifrt = x[JL_IT_NUM-1];
    wire Qfrt = y[JL_IT_NUM-1];
    wire ThetaR = z[JL_IT_NUM-1];

    genvar i;
    generate for(i=0;i<JL_IT_NUM-1;i=i+1) begin
      derotator uDerotator (clk_fs,!rstb,x[i],y[i],z[i],x[i+1],y[i+1],z[i+1]);
      defparam uDerotator.BW = BW;
      defparam UDerotator.ABW = ABW;
      defparam uDerotator.iteration = i;
      defparam uDerotator.tangle = tanangle(i);
    end 
    endgenerate

    
    // Filter 1
    wire [BW-1:0] Ilpf1,Qlpf1;
    reg  [BW-1:0] Idec1,Qdec1;

    fir #(.BW(BW),.N(5),.S(BW-1)) uIFIR1 (
        .CK(clk_fs),
        .RB(rstb),
        .X(Ifrt),
        .C({12'd211,12'd420,12'd503,12'd420,12'd211}),
        .Y(Ilpf1));
    
    fir #(.BW(BW),.N(5),.S(BW-1)) uQFIR1 (
        .CK(clk_fs),
        .RB(rstb),
        .X(Qfrt),
        .C({12'd211,12'd420,12'd503,12'd420,12'd211}),
        .Y(Qlpf1));

    //Decimate 1

    always_ff @(posedge clk_fso2 or negedge rstb)
        if (rstb==0)
         begin
            Idec1 <= 'd0;
            Qdec1 <= 'd0;
         end
        else
         begin
            Idec1 <= Ilpf1;
            Qdec1 <= Qlpf1;
         end

    //Filter 2

    
    wire [BW-1:0] Ilpf2,Qlpf2;
    reg  [BW-1:0] Idec2,Qdec2;

    fir #(.BW(BW),.N(5),.S(BW-1)) uIFIR2 (
        .CK(clk_fso2),
        .RB(rstb),
        .X(Idec1),
        .C({12'd211,12'd420,12'd503,12'd420,12'd211}),
        .Y(Illpf2));
    
    fir #(.BW(BW),.N(5),.S(BW-1)) uQFIR2 (
        .CK(clk_fs),
        .RB(rstb),
        .X(Qdec1),
        .C({12'd211,12'd420,12'd503,12'd420,12'd211}),
        .Y(Qlpf2));
    
    //Decimate 2


    always_ff @(posedge clk_fso4 or negedge rstb)
        if (rstb==0)
         begin
            Idec2 <= 'd0;
            Qdec2 <= 'd0;
         end
        else
         begin
            Idec2 <= Ilpf2;
            Qdec2 <= Qlpf2;
         end

    assign Qout = Qdec2;
    assign Iout = Idec2;

endmodule    

"""

"\n// Angle representation is always a 0.bbbbbb of a circle\n// 100000 -> half circle (pi)\n// 111111 -> 2*pi (almost)\n\n\nmodule FDC\n#(\n    parameter BW = 12,\n    parameter ABW = 8,\n)\n(\n    input clk_fs,  //  Sampling clock\n    input clk_fso2,  // Decimation clock fsample/2\n    input clk_fso4,  // Decimation clock fsample/4\n    input rstb, // Reset\n\n    input [BW-1:0] Iin,\n    input [BW-1:0] Qin,\n    input [ABW-1:0] Wif,\n    output [BW-1:0] Iout,\n    output [BW-1:0] Qout\n)\n    reg  [BW-1:0] Iin_s, Qin_s;\n    wire  [BW-1:0] Igr, Qgr;\n\n    // Sample\n    always_ff @(posedge clk_fs or negedge rstb)\n        if (rstb == 0)\n         begin\n            Iin_s <= 'd0;\n            Qin_s <= 'd0;\n         end\n        else\n         begin\n            Iin_s <= Iin;\n            Qin_s <= Qin;\n         end\n\n    \n    // Compute rotation angle theta \n    reg [ABW-1:0] theta; \n\n    always_ff @(posedge clk_fs or negedge rstb)\n        if (rstb==0)\n            theta <= 0

In [54]:
itNum = 8
abw = 10
itNumLd = log2(itNum) |> x->convert(Int,x)

#

tangleL = [
   atan(1/2^it)/pi/2*2^abw
    for it=0:(itNum-1)
    ] 
tangleLHD = [
   convert(Int,round(it))
    for it=tangleL
    ] 
error = sum(tangleLHD .- tangleL)

0.16168858105937822

In [62]:
tangleCase = [
    "\t\t\t $itNumLd'd$(i-1) : tangle = $abw'd$a;\n"
    for (i,a)=enumerate(tangleLHD)
] |> x->*(x...)
println(tangleCase)

sv_template_mod = replace(
    sv_template,
    r"parameter\s*ABW\s*=\s*\d+" => "parameter ABW = $abw"
)

sv_template_mod = replace(sv_template_mod,
    r"^\s*JL_FILL_TABLE"m=>tangleCase
)
sv_template_mod = replace(sv_template_mod,
    "JL_IT_NUM"=>"$itNum"
)
println(sv_template_mod)


			 3'd0 : tangle = 10'd128;
			 3'd1 : tangle = 10'd76;
			 3'd2 : tangle = 10'd40;
			 3'd3 : tangle = 10'd20;
			 3'd4 : tangle = 10'd10;
			 3'd5 : tangle = 10'd5;
			 3'd6 : tangle = 10'd3;
			 3'd7 : tangle = 10'd1;


// Angle representation is always a 0.bbbbbb of a circle
// 100000 -> half circle (pi)
// 111111 -> 2*pi (almost)


module FDC
#(
    parameter BW = 12,
    parameter ABW = 10,
)
(
    input clk_fs,  //  Sampling clock
    input clk_fso2,  // Decimation clock fsample/2
    input clk_fso4,  // Decimation clock fsample/4
    input rstb, // Reset

    input [BW-1:0] Iin,
    input [BW-1:0] Qin,
    input [ABW-1:0] Wif,
    output [BW-1:0] Iout,
    output [BW-1:0] Qout
)
    reg  [BW-1:0] Iin_s, Qin_s;
    wire  [BW-1:0] Igr, Qgr;

    // Sample
    always_ff @(posedge clk_fs or negedge rstb)
        if (rstb == 0)
         begin
            Iin_s <= 'd0;
            Qin_s <= 'd0;
         end
        else
         begin
            Iin_s <= Iin;
            Qin_s <= Q

In [64]:
open(f->write(f,sv_template_mod),"../rtl/FDC.sv","w")

3877